# Tutorial: Cell-Level LDSC analysis

This tutorial demonstrates how to perform cell-type-specific LD Score regression (LDSC) analysis through the `cellink` package. The `cellink` package provides a unified interface to LDSC and its preparation scripts, making it easier to perform comprehensive genetic analyses that identify which cell types are most relevant to complex traits and diseases.

This notebook assumes familiarity with single-cell data processing and basic statistical genetics concepts. The `cellink` package provides convenient wrapper functions that handle data preparation and formatting for LDSC. For LDSC installation, please follow instructions [here](https://github.com/bulik/ldsc). We recommend utilizing LDSC via a Docker image like [this one](https://hub.docker.com/r/zijingliu/ldsc). For usage on HPCs, please consider using singularity or enroot.

## Environment Setup

We begin by importing necessary libraries and defining key parameters for our analysis. The `cellink` package provides wrapper functions for LDSC that automatically handle preprocessing, data formatting and preparation.

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os

from cellink._core import DAnn, GAnn
from cellink.resources import get_onek1k
from cellink.tl.external import (
    preprocess_for_sldsc,
    generate_sldsc_genesets,
    generate_gene_coord_file,
    configure_ldsc_runner,
    make_annot_from_donor_data,
    munge_sumstats,
    estimate_ld_scores_from_donor_data,
    estimate_heritability,
    estimate_genetic_correlation,
    compute_ld_scores_with_annotations_from_donor_data,
    estimate_celltype_specific_heritability,
)
from cellink.resources import get_1000genomes_ld_scores, get_1000genomes_ld_weights
from cellink.resources import get_gwas_catalog_study_summary_stats

# Analysis parameters
chrom = 22
cell_type = "CD8 Naive"
celltype_key = "predicted.celltype.l2"
original_donor_col = "donor_id"

## Load and Prepare Data

We load the OneK1K dataset, which contains both genotype and single-cell expression data. We also add gene annotations from Ensembl for our analysis.

In [2]:
# Load the dataset
dd = get_onek1k(config_path="../../src/cellink/resources/config/onek1k.yaml", verify_checksum=False)
print(f"Dataset shape: {dd.shape}")


# Add gene annotations from Ensembl
def _get_ensembl_gene_id_start_end_chr():
    from pybiomart import Server

    server = Server(host="http://www.ensembl.org")
    dataset = server.marts["ENSEMBL_MART_ENSEMBL"].datasets["hsapiens_gene_ensembl"]
    ensembl_gene_id_start_end_chr = dataset.query(
        attributes=["ensembl_gene_id", "start_position", "end_position", "chromosome_name"]
    )
    ensembl_gene_id_start_end_chr = ensembl_gene_id_start_end_chr.set_index("Gene stable ID")
    ensembl_gene_id_start_end_chr = ensembl_gene_id_start_end_chr.rename(
        columns={
            "Gene start (bp)": GAnn.start,
            "Gene end (bp)": GAnn.end,
            "Chromosome/scaffold name": GAnn.chrom,
        }
    )
    return ensembl_gene_id_start_end_chr


ensembl_gene_id_start_end_chr = _get_ensembl_gene_id_start_end_chr()
dd.C.var = dd.C.var.join(ensembl_gene_id_start_end_chr)

# Set up donor information
dd.C.obs[DAnn.donor] = dd.C.obs[original_donor_col]

INFO:root:/Users/larnoldt/cellink_data/onek1k/onek1k_cellxgene.h5ad already exists
INFO:root:/Users/larnoldt/cellink_data/onek1k/OneK1K.noGP.vcf.gz already exists
INFO:root:/Users/larnoldt/cellink_data/onek1k/OneK1K.noGP.vcf.gz.csi already exists
INFO:root:/Users/larnoldt/cellink_data/onek1k/gene_counts_Ensembl_105_phenotype_metadata.tsv.gz already exists
/opt/miniconda3/envs/single_cell_base3/lib/python3.12/site-packages/pandas/core/internals/blocks.py:2588: RuntimeWarning: invalid value encountered in cast
  return self.values.astype(_dtype_obj)


Dataset shape: (981, 10595884, 1248980, 36469)


In [3]:
dd.G.obs["donor_id"] = dd.G.obs.index

### Cell-Type-Specific LDSC analysis

Cell-type-specific LDSC analysis helps identify which cell types are most relevant to complex traits by testing whether genetic variants associated with a trait are enriched in genes specifically expressed in certain cell types. This analysis follows the method described in [Duncan et al. 2025](https://www.nature.com/articles/s41593-024-01834-w).

#### Step 1: Preprocessing and Gene Set Generation
First, we preprocess the single-cell data to compute cell-type-specific gene expression and identify genes that are specifically expressed in each cell type.

In [ ]:
dd.C.var["gene"] = dd.C.var_names
adata = dd.C.copy()
adata_filtered, mean_expr, specificity = preprocess_for_sldsc(
    adata,
    celltype_col=celltype_key,
    gene_col="gene",
    gene_identifier_mode="ensembl",
    genome_build="GRCh38",
    inplace=False,
)

INFO:cellink.tl.external._sldsc_utils:Applying log1p transformation
INFO:cellink.tl.external._sldsc_utils:Log1p applied.
INFO:cellink.tl.external._sldsc_utils:Querying Ensembl BioMart (GRCh38)...
INFO:cellink.tl.external._sldsc_utils:Fetching gene annotations from GRCh38...
INFO:cellink.tl.external._sldsc_utils:Fetched annotations for 86371 genes from GRCh38
INFO:cellink.tl.external._sldsc_utils:Removing version suffixes from Gene IDs
INFO:cellink.tl.external._sldsc_utils:Dropping conflicting columns from adata.var before merge: ['chrom', 'start', 'end']
INFO:cellink.tl.external._sldsc_utils:Annotated 35522 / 36469 genes.
INFO:cellink.tl.external._sldsc_utils:Using annotation columns: gene=gene, biotype=gene_biotype, chr=chrom, start=start, end=end
INFO:cellink.tl.external._sldsc_utils:Applying gene filters
INFO:cellink.tl.external._sldsc_utils:Protein-coding genes: 19273
INFO:cellink.tl.external._sldsc_utils:Expressed genes: 31285
INFO:cellink.tl.external._sldsc_utils:Unique gene name

Next, we generate gene sets for each cell type containing the top 10% most specifically expressed genes. These gene sets will be used to create genomic annotations.

In [15]:
summary = generate_sldsc_genesets(specificity, dd.C, out_dir="ldsc_genesets", top_frac=0.10, overwrite=True)
summary

INFO:cellink.tl.external._sldsc_utils:Removing version suffixes from Gene IDs
INFO:cellink.tl.external._sldsc_utils:Writing gene sets to ldsc_genesets
INFO:cellink.tl.external._sldsc_utils:Overlapping genes: 18068/18068
INFO:cellink.tl.external._sldsc_utils:Selecting top 1807 genes (10.0%) per cell type
INFO:cellink.tl.external._sldsc_utils:Wrote control gene set with 18068 genes
INFO:cellink.tl.external._sldsc_utils:Generated 31 cell-type-specific gene sets


,cell_type,n_genes,output_path
0,ASDC,1807,ldsc_genesets/ASDC.GeneSet
1,B intermediate,1807,ldsc_genesets/B_intermediate.GeneSet
2,B memory,1807,ldsc_genesets/B_memory.GeneSet
3,B naive,1807,ldsc_genesets/B_naive.GeneSet
4,CD4 CTL,1807,ldsc_genesets/CD4_CTL.GeneSet
5,CD4 Naive,1807,ldsc_genesets/CD4_Naive.GeneSet
6,CD4 Proliferating,1807,ldsc_genesets/CD4_Proliferating.GeneSet
7,CD4 TCM,1807,ldsc_genesets/CD4_TCM.GeneSet
8,CD4 TEM,1807,ldsc_genesets/CD4_TEM.GeneSet
9,CD8 Naive,1807,ldsc_genesets/CD8_Naive.GeneSet


We also need to generate a gene coordinate file that maps genes to their genomic positions:

In [4]:
generate_gene_coord_file("gene_coords.txt", gene_identifier_mode="ensembl", genome_build="GRCh38", overwrite=True)

INFO:cellink.tl.external._sldsc_utils:Fetching gene annotations from Ensembl GRCh38...
INFO:cellink.tl.external._sldsc_utils:Querying Ensembl BioMart (GRCh38)...
INFO:cellink.tl.external._sldsc_utils:Fetching gene annotations from GRCh38...
INFO:cellink.tl.external._sldsc_utils:Fetched annotations for 86371 genes from GRCh38
INFO:cellink.tl.external._sldsc_utils:Removing version suffixes from gene identifiers
INFO:cellink.tl.external._sldsc_utils:Writing 86369 gene coordinates to gene_coords.txt
INFO:cellink.tl.external._sldsc_utils:Successfully created gene coordinate file: gene_coords.txt


#### Step 2: Configure LDSC Runner
Before running LDSC commands, we need to configure the runner. LDSC can be executed via a local installation or through container solutions like Docker or Singularity. Sample configuration files are provided in `./src/cellink/tl/external/config/` for local execution, Docker, and Singularity.

In [6]:
runner = configure_ldsc_runner(config_path="../../src/cellink/tl/external/config/ldsc_docker.yaml")

#### Step 3: Prepare Data for Analysis
To speed up computation in this tutorial, we filter the data to a specific chromosome and a random subset of SNPs. Note: In a real analysis, you would process all chromosomes without subsetting.

In [4]:
np.random.seed(42)
all_selected_idx = []
for chrom in range(1, 23):
    chrom_idx = np.where(dd.G.var.chrom == str(chrom))[0]
    n_snps = max(1, int(len(chrom_idx) * 0.001))
    selected_idx = np.random.choice(chrom_idx, n_snps, replace=False)
    all_selected_idx.extend(selected_idx)
all_selected_idx = np.sort(all_selected_idx)

dd = dd[:, all_selected_idx, :, :].copy()
dd

╔═ DonorData(n_donors=981, n_cells_per_donor=[333-3,511], donor_id='donor_id') ═══════════════════════════════╗
║ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ ║
║ ┃ G (donors)                                         ┃ C (cells)                                          ┃ ║
║ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ ║
║ │ AnnData object with n_obs × n_vars = 981 × 10,299  │ AnnData object with n_obs × n_vars = 1,248,980 ×   │ ║
║ │                                                    │ 36,469                                             │ ║
║ │     obs: 'donor_id'                                │     obs: 'orig.ident', 'nCount_RNA',               │ ║
║ │                                                    │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │ ║
║ │                                                    │ 'pool_number', 'predicted.celltype.l2',            │ ║
║ │                                                    │ 'predicted.celltype.l2.score', 'age',              │ ║
║ │                                                    │ 'organism_ontology_term_id',                       │ ║
║ │                                                    │ 'tissue_ontology_term_id',                         │ ║
║ │                                                    │ 'assay_ontology_term_id',                          │ ║
║ │                                                    │ 'disease_ontology_term_id',                        │ ║
║ │                                                    │ 'cell_type_ontology_term_id',                      │ ║
║ │                                                    │ 'self_reported_ethnicity_ontology_term_id',        │ ║
║ │                                                    │ 'development_stage_ontology_term_id',              │ ║
║ │                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │ ║
║ │                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │ ║
║ │                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │ ║
║ │                                                    │ 'self_reported_ethnicity', 'development_stage',    │ ║
║ │                                                    │ 'observation_joinid'                               │ ║
║ │     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     var: 'vst.mean', 'vst.variance',               │ ║
║ │ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'vst.variance.expected',                           │ ║
║ │ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'vst.variance.standardized', 'vst.variable',       │ ║
║ │ 'TYPED', 'TYPED_ONLY', 'id', 'id_mask', 'length',  │ 'feature_is_filtered', 'feature_name',             │ ║
║ │ 'quality', 'pos_hg19', 'id_hg19'                   │ 'feature_reference', 'feature_biotype',            │ ║
║ │                                                    │ 'feature_length', 'feature_type', 'start', 'end',  │ ║
║ │                                                    │ 'chrom'                                            │ ║
║ │     uns: 'kinship'                                 │     uns: 'cell_type_ontology_term_id_colors',      │ ║
║ │                                                    │ 'citation', 'default_embedding',                   │ ║
║ │                                                    │ 'schema_reference', 'schema_version', 'title'      │ ║
║ │     obsm: 'gPCs'                                   │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │ ║
║ │                                                    │ 'X_harmony', 'X_pca', 'X_umap'                     │ ║
║ │     varm: 'filter'                                 │     varm: 'PCs'                                    │ ║
║ └────────────────────────────────────────────────────┴────────────────────────

#### Step 4: Create Cell-Type Annotations
Now we create binary annotation files that indicate which SNPs are near cell-type-specific genes. This is done using LDSC's `make_annot` functionality, wrapped by `cellink`. We process two cell types (CD8 Naive and CD4 Naive) across all chromosomes.

In [8]:
for chrom in range(1, 23):
    for cell_type in ["CD8 Naive", "CD4 Naive"]:
        dd_chrom = dd.sel(G_var=dd.G.var.chrom == str(chrom), C_var=dd.C.var.chrom == str(chrom)).copy()
        result = make_annot_from_donor_data(
            dd=dd_chrom,
            annot_file=f"{cell_type.replace(" ", "_")}_{chrom}.annot.gz",
            gene_set_file=f"./ldsc_genesets/{cell_type.replace(" ", "_")}.GeneSet",
            gene_coord_file="gene_coords.txt",
            windowsize=100000,
            runner=runner,
        )

INFO:cellink.tl.external._ldsc:Exporting genotype data to PLINK format for annotation creation
Writing BED: 100%|██████████| 1/1 [00:07<00:00,  7.83s/it]

Writing FAM... done.
Writing BIM... done.



INFO:cellink.tl.external._ldsc:Creating annotation file: /ldsc/make_annot.py --bimfile ldsc_annot.bim --annot-file CD8_Naive_1.annot.gz --gene-set-file ./ldsc_genesets/CD8_Naive.GeneSet --gene-coord-file gene_coords.txt --windowsize 100000
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/make_annot.py --bimfile ldsc_annot.bim --annot-file CD8_Naive_1.annot.gz --gene-set-file ./ldsc_genesets/CD8_Naive.GeneSet --gene-coord-file gene_coords.txt --windowsize 100000
INFO:cellink.tl.external._ldsc:making gene set bed file
making annot file


INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.bim
INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.fam
INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.bed
INFO:cellink.tl.external._ldsc:Exporting genotype data to PLINK format for annotation creation
Writing BED: 100%|██████████| 1/1 [0

Writing FAM... done.
Writing BIM... done.



INFO:cellink.tl.external._ldsc:Creating annotation file: /ldsc/make_annot.py --bimfile ldsc_annot.bim --annot-file CD4_Naive_1.annot.gz --gene-set-file ./ldsc_genesets/CD4_Naive.GeneSet --gene-coord-file gene_coords.txt --windowsize 100000
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/make_annot.py --bimfile ldsc_annot.bim --annot-file CD4_Naive_1.annot.gz --gene-set-file ./ldsc_genesets/CD4_Naive.GeneSet --gene-coord-file gene_coords.txt --windowsize 100000
INFO:cellink.tl.external._ldsc:making gene set bed file
making annot file


INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.bim
INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.fam
INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.bed
INFO:cellink.tl.external._ldsc:Exporting genotype data to PLINK format for annotation creation
Writing BED: 100%|██████████| 1/1 [0

...


INFO:cellink.tl.external._ldsc:Creating annotation file: /ldsc/make_annot.py --bimfile ldsc_annot.bim --annot-file CD4_Naive_21.annot.gz --gene-set-file ./ldsc_genesets/CD4_Naive.GeneSet --gene-coord-file gene_coords.txt --windowsize 100000
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/make_annot.py --bimfile ldsc_annot.bim --annot-file CD4_Naive_21.annot.gz --gene-set-file ./ldsc_genesets/CD4_Naive.GeneSet --gene-coord-file gene_coords.txt --windowsize 100000
INFO:cellink.tl.external._ldsc:making gene set bed file
making annot file


INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.bim
INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.fam
INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.bed
INFO:cellink.tl.external._ldsc:Exporting genotype data to PLINK format for annotation creation
Writing BED: 100%|██████████| 1/1 

Writing FAM... done.
Writing BIM... done.



INFO:cellink.tl.external._ldsc:Creating annotation file: /ldsc/make_annot.py --bimfile ldsc_annot.bim --annot-file CD8_Naive_22.annot.gz --gene-set-file ./ldsc_genesets/CD8_Naive.GeneSet --gene-coord-file gene_coords.txt --windowsize 100000
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/make_annot.py --bimfile ldsc_annot.bim --annot-file CD8_Naive_22.annot.gz --gene-set-file ./ldsc_genesets/CD8_Naive.GeneSet --gene-coord-file gene_coords.txt --windowsize 100000
INFO:cellink.tl.external._ldsc:making gene set bed file
making annot file


INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.bim
INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.fam
INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.bed
INFO:cellink.tl.external._ldsc:Exporting genotype data to PLINK format for annotation creation
Writing BED: 100%|██████████| 1/1 

Writing FAM... done.
Writing BIM... done.



INFO:cellink.tl.external._ldsc:Creating annotation file: /ldsc/make_annot.py --bimfile ldsc_annot.bim --annot-file CD4_Naive_22.annot.gz --gene-set-file ./ldsc_genesets/CD4_Naive.GeneSet --gene-coord-file gene_coords.txt --windowsize 100000
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/make_annot.py --bimfile ldsc_annot.bim --annot-file CD4_Naive_22.annot.gz --gene-set-file ./ldsc_genesets/CD4_Naive.GeneSet --gene-coord-file gene_coords.txt --windowsize 100000
INFO:cellink.tl.external._ldsc:making gene set bed file
making annot file


INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.bim
INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.fam
INFO:cellink.tl.external._ldsc:Cleaned up file: ldsc_annot.bed


#### Step 5: Compute Cell-Type-Specific LD Scores
With annotations created, we now compute LD scores that incorporate cell-type-specific information. These LD scores quantify how much genetic variation near cell-type-specific genes contributes to linkage disequilibrium patterns.  We here use the function `compute_ld_scores_with_annotations_from_donor_data`. Instead, one could also perform this using 1000G plink data via `compute_ld_scores_with_annotations_from_bimfile` (Plink files may be downloaded via `cellink.resources.get_1000genomes_plink_files`). 

In [9]:
for chrom in range(1, 23):
    for cell_type in ["CD8 Naive", "CD4 Naive"]:
        print(f"Processing cell type: {cell_type}")

        dd_chrom = dd.sel(G_var=dd.G.var.chrom == str(chrom), C_var=dd.C.var.chrom == str(chrom)).copy()
        results = compute_ld_scores_with_annotations_from_donor_data(
            dd=dd_chrom,
            annot_file=f"{cell_type.replace(" ", "_")}_{chrom}.annot.gz",
            out_prefix=f"cts_ldscores_{cell_type.replace(' ', '_')}.{chrom}",
            run=True,
            runner=runner,
        )

Processing cell type: CD8 Naive


INFO:cellink.tl.external._ldsc:Exporting genotype data to PLINK format for LD score computation
Writing BED: 100%|██████████| 1/1 [00:10<00:00, 10.35s/it]

Writing FAM... done.
Writing BIM... done.



INFO:cellink.tl.external._ldsc:Computing LD scores with annotations: /ldsc/ldsc.py --l2 --bfile cts_ldscores_CD8_Naive.1 --annot CD8_Naive_1.annot.gz --out cts_ldscores_CD8_Naive.1 --ld-wind-cm 1.0 --thin-annot --yes-really
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/ldsc.py --l2 --bfile cts_ldscores_CD8_Naive.1 --annot CD8_Naive_1.annot.gz --out cts_ldscores_CD8_Naive.1 --ld-wind-cm 1.0 --thin-annot --yes-really
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./ldsc.py \
--ld-wind-cm 1.0 \
--out cts_ldscores_CD8_Nai

Processing cell type: CD4 Naive


...


INFO:cellink.tl.external._ldsc:Computing LD scores with annotations: /ldsc/ldsc.py --l2 --bfile cts_ldscores_CD8_Naive.22 --annot CD8_Naive_22.annot.gz --out cts_ldscores_CD8_Naive.22 --ld-wind-cm 1.0 --thin-annot --yes-really
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/ldsc.py --l2 --bfile cts_ldscores_CD8_Naive.22 --annot CD8_Naive_22.annot.gz --out cts_ldscores_CD8_Naive.22 --ld-wind-cm 1.0 --thin-annot --yes-really
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./ldsc.py \
--ld-wind-cm 1.0 \
--out cts_ldscores_C

Processing cell type: CD4 Naive


INFO:cellink.tl.external._ldsc:Exporting genotype data to PLINK format for LD score computation
Writing BED: 100%|██████████| 1/1 [00:02<00:00,  2.40s/it]

Writing FAM... done.
Writing BIM... done.



INFO:cellink.tl.external._ldsc:Computing LD scores with annotations: /ldsc/ldsc.py --l2 --bfile cts_ldscores_CD4_Naive.22 --annot CD4_Naive_22.annot.gz --out cts_ldscores_CD4_Naive.22 --ld-wind-cm 1.0 --thin-annot --yes-really
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/ldsc.py --l2 --bfile cts_ldscores_CD4_Naive.22 --annot CD4_Naive_22.annot.gz --out cts_ldscores_CD4_Naive.22 --ld-wind-cm 1.0 --thin-annot --yes-really
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./ldsc.py \
--ld-wind-cm 1.0 \
--out cts_ldscores_C

#### Step 6: Prepare Reference LD Scores and Weights
For the final analysis, we need baseline LD scores and regression weights. These control for genomic confounders and ensure proper statistical inference. We download these from the 1000 Genomes reference panel.

In [10]:
ldscores_path, ldscores_prefix = get_1000genomes_ld_scores(
    config_path="../../src/cellink/resources/config/1000genomes.yaml", population="EUR", return_path=True
)
ldweights_path, ldweights_prefix = get_1000genomes_ld_weights(
    config_path="../../src/cellink/resources/config/1000genomes.yaml", population="EUR", return_path=True
)

INFO:root:/Users/larnoldt/cellink_data/1000genomes_ld_scores_EUR/1000G_Phase3_baselineLD_v2.2_ldscores.tgz already exists
INFO:root:/Users/larnoldt/cellink_data/1000genomes_ld_weights_EUR/1000G_Phase3_weights_hm3_no_MHC.tgz already exists


Create a control file listing all cell-type-specific LD score prefixes:

In [11]:
with open("celltype_ldscores.txt", "w") as f:
    f.write("CD8_Naive\tcts_ldscores_CD8_Naive.\n")  # ,{os.path.join(ldscores_path, ldscores_prefix)}
    f.write("CD4_Naive\tcts_ldscores_CD4_Naive.\n")

We also need baseline LD scores. Here we compute them from our donor data (in a real analysis, you would use pre-computed baseline LD scores):

In [14]:
for chrom in range(1, 23):
    dd_chrom = dd.sel(G_var=dd.G.var.chrom == str(chrom), C_var=dd.C.var.chrom == str(chrom)).copy()
    result = estimate_ld_scores_from_donor_data(dd=dd_chrom, out_prefix=f"BaselineLD.{chrom}", run=True, runner=runner)

INFO:cellink.tl.external._ldsc:Exporting genotype data to PLINK format for LD score estimation
Writing BED: 100%|██████████| 1/1 [00:11<00:00, 11.06s/it]

Writing FAM... done.
Writing BIM... done.



INFO:cellink.tl.external._ldsc:Estimating LD scores: /ldsc/ldsc.py --bfile BaselineLD.1 --l2 --out BaselineLD.1 --ld-wind-cm 1.0 --yes-really
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/ldsc.py --bfile BaselineLD.1 --l2 --out BaselineLD.1 --ld-wind-cm 1.0 --yes-really
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./ldsc.py \
--ld-wind-cm 1.0 \
--out BaselineLD.1 \
--bfile BaselineLD.1 \
--yes-really  \
--l2  

Beginning analysis at Thu Nov  6 21:09:35 2025
Read list of 796 SNPs from BaselineLD.1.bim
Read list of 98

Writing FAM... done.
Writing BIM... done.



INFO:cellink.tl.external._ldsc:Estimating LD scores: /ldsc/ldsc.py --bfile BaselineLD.2 --l2 --out BaselineLD.2 --ld-wind-cm 1.0 --yes-really
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/ldsc.py --bfile BaselineLD.2 --l2 --out BaselineLD.2 --ld-wind-cm 1.0 --yes-really
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./ldsc.py \
--ld-wind-cm 1.0 \
--out BaselineLD.2 \
--bfile BaselineLD.2 \
--yes-really  \
--l2  

Beginning analysis at Thu Nov  6 21:10:01 2025
Read list of 864 SNPs from BaselineLD.2.bim
Read list of 98

...


INFO:cellink.tl.external._ldsc:Estimating LD scores: /ldsc/ldsc.py --bfile BaselineLD.20 --l2 --out BaselineLD.20 --ld-wind-cm 1.0 --yes-really
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/ldsc.py --bfile BaselineLD.20 --l2 --out BaselineLD.20 --ld-wind-cm 1.0 --yes-really
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./ldsc.py \
--ld-wind-cm 1.0 \
--out BaselineLD.20 \
--bfile BaselineLD.20 \
--yes-really  \
--l2  

Beginning analysis at Thu Nov  6 21:16:48 2025
Read list of 227 SNPs from BaselineLD.20.bim
Read lis

Writing FAM... done.
Writing BIM... done.



INFO:cellink.tl.external._ldsc:Estimating LD scores: /ldsc/ldsc.py --bfile BaselineLD.21 --l2 --out BaselineLD.21 --ld-wind-cm 1.0 --yes-really
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/ldsc.py --bfile BaselineLD.21 --l2 --out BaselineLD.21 --ld-wind-cm 1.0 --yes-really
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./ldsc.py \
--ld-wind-cm 1.0 \
--out BaselineLD.21 \
--bfile BaselineLD.21 \
--yes-really  \
--l2  

Beginning analysis at Thu Nov  6 21:17:04 2025
Read list of 135 SNPs from BaselineLD.21.bim
Read lis

Writing FAM... done.
Writing BIM... done.



INFO:cellink.tl.external._ldsc:Estimating LD scores: /ldsc/ldsc.py --bfile BaselineLD.22 --l2 --out BaselineLD.22 --ld-wind-cm 1.0 --yes-really
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/ldsc.py --bfile BaselineLD.22 --l2 --out BaselineLD.22 --ld-wind-cm 1.0 --yes-really
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./ldsc.py \
--ld-wind-cm 1.0 \
--out BaselineLD.22 \
--bfile BaselineLD.22 \
--yes-really  \
--l2  

Beginning analysis at Thu Nov  6 21:17:19 2025
Read list of 136 SNPs from BaselineLD.22.bim
Read lis

#### Step 7: Prepare GWAS Summary Statistics
For this demonstration, we generate fake GWAS summary statistics. In a real analysis, you would use actual GWAS summary statistics from published studies.

In [24]:
def generate_fake_sumstats(dd, output_file="fake_munged.sumstats.gz", subset_frac=0.8, n_extra_snps=1000, seed=42):
    np.random.seed(seed)

    n_real = int(len(dd.G.var) * subset_frac)
    real_idx = np.random.choice(len(dd.G.var), n_real, replace=False)

    snp_ids = dd.G.var.index[real_idx].tolist()
    allele_0 = dd.G.var["a0"].iloc[real_idx].tolist()
    allele_1 = dd.G.var["a1"].iloc[real_idx].tolist()

    for i in range(n_extra_snps):
        chrom = np.random.randint(1, 23)
        pos = np.random.randint(1000000, 50000000)
        ref = np.random.choice(["A", "C", "G", "T"])
        alt = np.random.choice([a for a in ["A", "C", "G", "T"] if a != ref])
        snp_ids.append(f"{chrom}_{pos}_{ref}_{alt}")
        allele_0.append(ref)
        allele_1.append(alt)

    z_scores = np.random.randn(len(snp_ids))
    large_effect_idx = np.random.choice(len(snp_ids), int(len(snp_ids) * 0.01), replace=False)
    z_scores[large_effect_idx] = np.random.randn(len(large_effect_idx)) * 3

    fake_sumstats = pd.DataFrame({"SNP": snp_ids, "A1": allele_1, "A2": allele_0, "Z": z_scores, "N": 336924.0})

    fake_sumstats.to_csv(output_file, sep="\t", index=False, compression="gzip", float_format="%.3f")
    print(f"Generated {len(fake_sumstats)} SNPs ({n_real} real, {n_extra_snps} fake) -> {output_file}")
    return output_file


generate_fake_sumstats(dd, subset_frac=0.8, n_extra_snps=1000)

Generated 9239 SNPs (8239 real, 1000 fake) -> fake_munged.sumstats.gz


'fake_munged.sumstats.gz'

#### Step 8: Run Cell-Type-Specific Heritability Analysis
Finally, we run the cell-type-specific heritability analysis, which tests whether each cell type shows enrichment for trait heritability.

In [26]:
result = estimate_celltype_specific_heritability(
    sumstats_file="fake_munged.sumstats.gz",
    ref_ld_chr="BaselineLD.",
    w_ld_chr="BaselineLD.",
    ref_ld_chr_cts="celltype_ldscores.txt",
    out_prefix="CHD_CD8_Naive_h2",
    run=True,
    runner=runner,
)

INFO:cellink.tl.external._ldsc:Running cell-type-specific heritability analysis: /ldsc/ldsc.py --h2-cts fake_munged.sumstats.gz --ref-ld-chr BaselineLD. --w-ld-chr BaselineLD. --ref-ld-chr-cts celltype_ldscores.txt --out CHD_CD8_Naive_h2
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/ldsc.py --h2-cts fake_munged.sumstats.gz --ref-ld-chr BaselineLD. --w-ld-chr BaselineLD. --ref-ld-chr-cts celltype_ldscores.txt --out CHD_CD8_Naive_h2
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./ldsc.py \
--h2-cts fake_munged.sumstats.

### Standard LDSC Analyses
Beyond cell-type-specific analysis, LDSC can also be used for standard heritability estimation and genetic correlation analysis. Here we demonstrate these capabilities.
#### SNP Heritability Estimation
First, we download real GWAS summary statistics from the GWAS Catalog and prepare them for LDSC analysis.

In [ ]:
runner = configure_ldsc_runner(config_path="../../src/cellink/tl/external/config/ldsc_docker.yaml")

In [36]:
gwas_summary_statistic_path_1 = get_gwas_catalog_study_summary_stats("GCST004787", return_path=True)
gwas_summary_statistic_path_1

INFO:root:Fetching https://www.ebi.ac.uk/gwas/rest/api/v2/studies/GCST004787
INFO:root:Found harmonised file: 28714975-GCST004787-EFO_0001645.h.tsv.gz
INFO:root:Downloading http://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/GCST004001-GCST005000/GCST004787/harmonised/28714975-GCST004787-EFO_0001645.h.tsv.gz to /Users/larnoldt/cellink_data/GCST004787_summary_stats.tsv.gz


PosixPath('/Users/larnoldt/cellink_data/GCST004787_summary_stats.tsv.gz')

LDSC requires "munged" (cleaned and standardized) summary statistics. The `munge_sumstats` function performs quality control, standardizes column names, and prepares the data:

In [7]:
munged_file_1 = munge_sumstats(
    sumstats_file=gwas_summary_statistic_path_1,
    out_prefix=str(Path(Path(gwas_summary_statistic_path_1).stem).stem + "_munged"),
    info_min=0.9,
    maf_min=0.01,
    signed_sumstats=("logor", 0),
    run=True,
    p_col="p_value",
    snp_col="variant_id",
    a1_col="effect_allele",
    a2_col="other_allele",
    n_samples=336924,
    runner=runner,
)

INFO:cellink.tl.external._ldsc:Running munge_sumstats: /ldsc/munge_sumstats.py --sumstats /Users/larnoldt/cellink_data/GCST004787_summary_stats.tsv.gz --out GCST004787_summary_stats_munged --N 336924 --signed-sumstats logor,0 --p p_value --a1 effect_allele --a2 other_allele --snp variant_id
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/munge_sumstats.py --sumstats /cellink_data/GCST004787_summary_stats.tsv.gz --out GCST004787_summary_stats_munged --N 336924 --signed-sumstats logor,0 --p p_value --a1 effect_allele --a2 other_allele --snp variant_id
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3


Download reference LD scores and weights for heritability estimation:

In [2]:
ldscores_path, ldscores_prefix = get_1000genomes_ld_scores(
    config_path="../../src/cellink/resources/config/1000genomes.yaml", population="EUR", return_path=True
)
ldweights_path, ldweights_prefix = get_1000genomes_ld_weights(
    config_path="../../src/cellink/resources/config/1000genomes.yaml", population="EUR", return_path=True
)

INFO:root:/Users/larnoldt/cellink_data/1000genomes_ld_scores_EUR/1000G_Phase3_baselineLD_v2.2_ldscores.tgz already exists


INFO:root:Downloading https://zenodo.org/records/10515792/files/1000G_Phase3_weights_hm3_no_MHC.tgz?download=1 to /Users/larnoldt/cellink_data/1000genomes_ld_weights_EUR/1000G_Phase3_weights_hm3_no_MHC.tgz


Now estimate SNP heritability:

In [16]:
h2_results = estimate_heritability(
    sumstats_file=str(Path(Path(gwas_summary_statistic_path_1).stem).stem + "_munged.sumstats.gz"),
    ref_ld_chr=os.path.join(ldscores_path, ldscores_prefix),
    w_ld_chr=os.path.join(ldweights_path, ldweights_prefix),
    out_prefix=str(Path(Path(gwas_summary_statistic_path_1).stem).stem + "_h2"),
    run=True,
    runner=runner,
)

INFO:cellink.tl.external._ldsc:Estimating heritability: /ldsc/ldsc.py --h2 GCST004787_summary_stats_munged.sumstats.gz --ref-ld-chr /Users/larnoldt/cellink_data/1000genomes_ld_scores_EUR/baselineLD. --w-ld-chr /Users/larnoldt/cellink_data/1000genomes_ld_weights_EUR/weights.hm3_noMHC. --out GCST004787_summary_stats_h2
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/ldsc.py --h2 GCST004787_summary_stats_munged.sumstats.gz --ref-ld-chr /cellink_data/1000genomes_ld_scores_EUR/baselineLD. --w-ld-chr /cellink_data/1000genomes_ld_weights_EUR/weights.hm3_noMHC. --out GCST004787_summary_stats_h2
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathem

#### Genetic Correlation Analysis
Genetic correlation analysis quantifies the shared genetic basis between two traits. We download a second GWAS for coronary artery disease and compute the genetic correlation.

In [27]:
gwas_summary_statistic_path_2 = get_gwas_catalog_study_summary_stats("GCST90043957", return_path=True)
gwas_summary_statistic_path_2

INFO:root:Fetching https://www.ebi.ac.uk/gwas/rest/api/v2/studies/GCST90043957
INFO:root:Found harmonised file: 34737426-GCST90043957-MONDO_0021661.h.tsv.gz
INFO:root:Downloading http://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/GCST90043001-GCST90044000/GCST90043957/harmonised/34737426-GCST90043957-MONDO_0021661.h.tsv.gz to /Users/larnoldt/cellink_data/GCST90043957_summary_stats.tsv.gz


PosixPath('/Users/larnoldt/cellink_data/GCST90043957_summary_stats.tsv.gz')

In [34]:
munged_file_2 = munge_sumstats(
    sumstats_file=gwas_summary_statistic_path_2,
    out_prefix=str(Path(Path(gwas_summary_statistic_path_2).stem).stem + "_munged"),
    info_min=0.9,
    maf_min=0.01,
    signed_sumstats=("beta", 0),
    run=True,
    p_col="p_value",
    snp_col="variant_id",
    a1_col="effect_allele",
    a2_col="other_allele",
    n_samples=456348,
    runner=runner,
)

INFO:cellink.tl.external._ldsc:Running munge_sumstats: /ldsc/munge_sumstats.py --sumstats /Users/larnoldt/cellink_data/GCST90043957_summary_stats.tsv.gz --out GCST90043957_summary_stats_munged --N 456348 --signed-sumstats beta,0 --p p_value --a1 effect_allele --a2 other_allele --snp variant_id
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/munge_sumstats.py --sumstats /cellink_data/GCST90043957_summary_stats.tsv.gz --out GCST90043957_summary_stats_munged --N 456348 --signed-sumstats beta,0 --p p_value --a1 effect_allele --a2 other_allele --snp variant_id
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public Licen

Finally, estimate the genetic correlation between the two traits:

In [37]:
rg_results = estimate_genetic_correlation(
    sumstats_files=[
        str(Path(Path(gwas_summary_statistic_path_1).stem).stem + "_munged.sumstats.gz"),
        str(Path(Path(gwas_summary_statistic_path_2).stem).stem + "_munged.sumstats.gz"),
    ],
    ref_ld_chr=os.path.join(ldscores_path, ldscores_prefix),
    w_ld_chr=os.path.join(ldweights_path, ldweights_prefix),
    out_prefix="CHD_rg",
    run=True,
    runner=runner,
)

INFO:cellink.tl.external._ldsc:Estimating genetic correlation: /ldsc/ldsc.py --rg GCST004787_summary_stats_munged.sumstats.gz,GCST90043957_summary_stats_munged.sumstats.gz --ref-ld-chr /Users/larnoldt/cellink_data/1000genomes_ld_scores_EUR/baselineLD. --w-ld-chr /Users/larnoldt/cellink_data/1000genomes_ld_weights_EUR/weights.hm3_noMHC. --out CHD_rg
INFO:cellink.tl.external._ldsc:Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data zijingliu/ldsc /ldsc/ldsc.py --rg GCST004787_summary_stats_munged.sumstats.gz,GCST90043957_summary_stats_munged.sumstats.gz --ref-ld-chr /cellink_data/1000genomes_ld_scores_EUR/baselineLD. --w-ld-chr /cellink_data/1000genomes_ld_weights_EUR/weights.hm3_noMHC. --out CHD_rg
INFO:cellink.tl.external._ldsc:*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broa

# Summary
This tutorial demonstrated how to perform comprehensive LDSC analyses using the `cellink` package, including:

1. Cell-type-specific heritability analysis: Identifying which cell types are most relevant to complex traits
2. SNP heritability estimation: Quantifying the proportion of trait variance explained by common genetic variants
3. Genetic correlation analysis: Measuring shared genetic architecture between traits

The `cellink` package simplifies these analyses by providing unified wrapper functions that handle data formatting, file management, and command execution for LDSC and its auxiliary tools.